<a href="https://colab.research.google.com/github/Buuuuli/AIPI540_individual/blob/main/try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT, force_remount=False)

Mounted at /content/drive


In [ ]:
#PROJ ="MyDrive/repos"
#PROJECT_PATH = join(ROOT, PROJ)
#!mkdir "PROJECT_PATH"

In [2]:
!ls

drive  sample_data


In [3]:
%cd /content/drive/"My Drive"/AIPI540

/content/drive/My Drive/AIPI540


In [9]:
import os
import urllib.request
import zipfile
import copy
import time
import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms
import torchvision
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchsummary import summary
import cv2 as cv
import glob
from PIL import Image
import pickle as pkl

In [5]:
# check what types of image we get
#set the image directory
files = os.listdir("./image/")
# print the types of image
print(set([x.split(".")[1] for x in files]))


{'jpg'}


In [6]:
# prepare the directory for each image type
extensions = ("*.jpg")
path = []
for extension in extensions:
    path.extend(glob.glob("./image/"+extension))

In [ ]:
# read data
cv_img = []
for img in path:
    # read image > to Tensor > permute the data > resize > flatten
    n = cv.imread(img)
    cv_img.append(n)

In [1]:
cv_img[2]

NameError: ignored

In [ ]:
import pickle as pkl

with open('./read_imgs.pkl','wb') as fp:
  pkl.dump(cv_img,fp)